## Домашняя работа
Группа: Т12О-101М-20

Студент: Гриньков Владислав Леонидович

### Задача 1: применяем PCA-трансформацию

Модифицируйте файл train.py - добавьте в пайплайн обучения модели сжатие размерности до n_components=2 с помощью PCA и обучите модель в докере на "сжатых" данных. Сохраните полученный объект pca_transformer.pkl, который умеет выполнять сжатие данных.

Решением домашки считается модифицированный файл train.py

In [7]:
# --- ВАШ КОД ТУТ --
import pickle
import os
import logging
from pathlib import Path

import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA

LOG_FORMAT = '%(asctime)s | %(levelname)-8s | %(filename)-25.25s:%(lineno)-4d | %(message)s'

# NOTE: отключил логирование в файл при запуске в jupyter
# log_filename = "/www/classifier/data/service.log"
logging.basicConfig(level=logging.INFO, format=LOG_FORMAT)

def load_data(from_file: Path):
    data_source = np.genfromtxt(from_file.resolve().as_posix(), delimiter=',', skip_header=1)
    X = data_source[:, :3]
    y = data_source[:, 3]
    
    return X, y

def transformer(X, save_into: Path):
    pca_transformer = PCA(n_components=2).fit(X)
    X_pca = pca_transformer.transform(X)
    
    with save_into.open('wb') as f:
        pickle.dump(pca_transformer, f)
        logging.info('T-SNE Transformer обучен и сохранен в %s' % save_into.resolve())
        
    return X_pca


def classifier(X, y, save_into: Path): 
    clf = DecisionTreeClassifier(max_depth=3, random_state=42)
    clf.fit(X, y)

    with open('clf.pkl', 'wb') as f:
        pickle.dump(clf, f)
        logging.info('Модель обучена и сохранена в %s' % save_into.resolve())


def pipeline():
    X, y = load_data(Path('data/client_segmentation.csv'))
    X = transformer(X, save_into=Path('./pca_transformer.pkl'))

    classifier(X, y, save_into=Path('./clf.pkl'))
    

pipeline()
# ------------------

2021-04-25 16:46:45,878 | INFO     | <ipython-input-7-625032a3:30   | T-SNE Transformer обучен и сохранен в /home/cdayz/Sources/university/data_management_homeworks/jupyter/pca_transformer.pkl
2021-04-25 16:46:45,881 | INFO     | <ipython-input-7-625032a3:41   | Модель обучена и сохранена в /home/cdayz/Sources/university/data_management_homeworks/jupyter/clf.pkl


### Домашнее задание: трансформация входных фичей на лету

Модифицируйте файл `service.py`: добавьте загрузку объекта для трансформации `pca_tansformer.pkl` и применяйте её **в докере** для трансформации набора входных фич в сжатые:
<pre>
[x1, x2, x3] -> [x1_pca, x2_pca]
</pre>

Соответственно, predict надо выполнять на *сжатых* фичах


Решением домашки считается модифицированный файл *service.py*

In [8]:
# --- ВАШ КОД ТУТ --
"""
Умеет выполнять классификацию клиентов по трём фичам
Запускаем из python3:
    python3 service.py
Проверяем работоспособность:
    curl http://127.0.0.1:5000/
"""

import copy
import json
import http.server
import logging
import os
import pickle
import socketserver
import sys
from http import HTTPStatus
from re import compile

import numpy as np
from sklearn.tree import DecisionTreeClassifier

# файл, куда посыпятся логи модели

LOG_FORMAT = '%(asctime)s | %(levelname)-8s | %(filename)-25.25s:%(lineno)-4d | %(message)s'
logging.basicConfig(level=logging.INFO, format=LOG_FORMAT)


def parse_params(params) -> dict:
    """
        Выдираем параметры из GET-запроса
    """
    params_list = params.split('&')
    params_dict = {'x1': None, 'x2': None, 'x3': None}
    for param in params_list:
        key, value = param.split('=')
        params_dict[key] = float(value)
    return params_dict


class Handler(http.server.SimpleHTTPRequestHandler):
    """Простой http-сервер"""

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        
    def handle_ping(self) -> dict:
        return {'message': 'pong'}

    def handle_query(self, params: dict) -> dict:
        response = params
        used_features = self.reduce_features(params)
        predicted_class = int(classifier_model.predict(used_features)[0])
        
        logging.info('predicted_class %s' % predicted_class)
        response.update({'predicted_class': predicted_class})
        
        return response

    def reduce_features(self, features: dict) -> dict:
        return transformer.transform(np.array([[features['x1'], features['x2'], features['x3']]]))

        
    def get_response(self) -> dict:
        """Пример запроса
        
        http://0.0.0.0:5000/classifier/?x1=1&x2=-2.2&x3=1.05
        """
        response = {'ping': 'ok'}
        url_with_params = self.path.split('?')

        if len(url_with_params) == 2 and self.path.startswith('/classifier'):
            return self.handle_query(parse_params(url_with_params[1]))

        elif self.path.startswith('/ping/'):
            return self.handle_ping()

        return response

    def do_GET(self):
        # заголовки ответа
        self.send_response(HTTPStatus.OK)
        self.send_header("Content-type", "application/json")
        self.end_headers()
        self.wfile.write(json.dumps(self.get_response()).encode())


logging.info('Загружаем обученную модель')
with open('./clf.pkl', 'rb') as f:
    classifier_model = pickle.load(f)
    logging.info('Модель загружена: %s' % classifier_model)

with open('./pca_transformer.pkl', 'rb') as f:
    transformer = pickle.load(f)
    logging.info('Модель загружена: %s' % transformer)


# NOTE: disbale for not start server in jupyter
# if __name__ == '__main__':
#     logging.info('Server started')
#     classifier_service = socketserver.TCPServer(('', 5000), Handler)
#     classifier_service.serve_forever()

# ------------------

2021-04-25 16:46:51,773 | INFO     | <ipython-input-8-e5306d3d:90   | Загружаем обученную модель
2021-04-25 16:46:51,774 | INFO     | <ipython-input-8-e5306d3d:93   | Модель загружена: DecisionTreeClassifier(max_depth=3, random_state=42)
2021-04-25 16:46:51,776 | INFO     | <ipython-input-8-e5306d3d:97   | Модель загружена: PCA(n_components=2)


### Домашнее задание: Используем Flask

Перепишите сервис на использование Flask. Вы можете взять готовый базовый образ с Flask, либо добавить установку в тот контейнер, который есть - это нужно сделать в Dockerfile


In [12]:
# --- ВАШ КОД ТУТ --
import logging
import pickle

from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/ping')
def ping():
    return jsonify({'message': 'pong'})


@app.route('/classifier')
def classifier():
    if not all(param in request.args for param in ('x1', 'x2', 'x3')):
        return jsonify({'error': 'query parameters x1, x2, x3 required'})

    try:
        x1 = float(request.args['x1'])
        x2 = float(request.args['x2'])
        x3 = float(request.args['x3'])
    except (TypeError, ValueError):
        logging.error('bad request format')
        return jsonify({'error': 'all of x1, x2, x3 query parameters must be float numbers'})

    try:
        used_features = transformer.transform(np.array([[x1, x2, x3]]))
        predicted_class = int(classifier_model.predict(used_features)[0])

        response = {'x1': x1, 'x2': x2, 'x3': x3, 'predicted_class': predicted_class}

        return jsonify(response)
    except Exception as err:
        logging.exception(f"can't predict class for parameters [{x1}, {x2}, {x3}]")
        return jsonify({'error': f'internal error: {repr(err)}'})


with open('./clf.pkl', 'rb') as f:
    classifier_model = pickle.load(f)
    logging.info('Модель загружена: %s' % classifier_model)

with open('./pca_transformer.pkl', 'rb') as f:
    transformer = pickle.load(f)
    logging.info('Модель загружена: %s' % transformer)

if __name__ == "__main__":
    LOG_FORMAT = '%(asctime)s | %(levelname)-8s | %(filename)-25.25s:%(lineno)-4d | %(message)s'
    logging.basicConfig(level=logging.INFO, format=LOG_FORMAT)

#     Disable app start in jupyter notebook
#     app.run(port=5001)
# ------------------

2021-04-25 16:49:26,168 | INFO     | <ipython-input-12-babc6e1:41   | Модель загружена: DecisionTreeClassifier(max_depth=3, random_state=42)
2021-04-25 16:49:26,170 | INFO     | <ipython-input-12-babc6e1:45   | Модель загружена: PCA(n_components=2)


## Домашнее задание: строим KNN

В реальной жизни KNN-рекомендатель не стоит делать на основе `sklearn.neighbors.NearestNeighbors` - есть готовые реализации, заточенные специально для построения рекомендательных систем. Хорошим примером такой реализации является [пакет implictit](). В рамках домашней работы предлагается разобраться с реализацией KNN-рекомендателя из этой библиотеки 

Почитайте документацию по модулю `implicit.nearest_neighbours.CosineRecommender`. Обучите KNN-рекомендатель и воспользуйтесь методом `recommend` для построения рекомендаций


In [4]:
import pandas as pd
import numpy as np

content_views = pd.read_csv(
    'recsys_data/content_views.zip', delimiter=',', header=0, compression='zip',
    names = ['user_id', 'content_id', 'view_duration', 'view_ts', 'dt', 'platform'],
    dtype = {'user_id': np.uint32, 'content_id': np.uint16, 'view_duration': np.uint16},
    parse_dates = [3, 4]
)


print('Количество просмотров %s' % content_views.user_id.count())

content_views.head(3)

Количество просмотров 489565


,user_id,content_id,view_duration,view_ts,dt,platform
0,4649,52867,735,2019-03-18 20:40:57+03:00,2019-03-18,LG
1,16,48800,361,2019-03-18 11:48:27+03:00,2019-03-18,LG
2,5380,47146,268,2019-02-17 13:06:33+03:00,2019-02-17,LG


In [5]:
content_description = pd.read_csv(
    'recsys_data/content_description.zip', delimiter=',', header=0, compression='zip',
    names = ['content_id', 'origin_country', 'release_date', 'kinopoisk_rating', 'compilation_id', 'genre'],
    dtype = {'content_id': np.uint16},
    parse_dates = [2]
)

print('Количество доступного контента %s' % content_description.content_id.count())

content_description.head(3)

Количество доступного контента 126182


,content_id,origin_country,release_date,kinopoisk_rating,compilation_id,genre
0,1974,87.0,2009-12-15,7.27,153,Для детей
1,2148,87.0,2009-12-21,7.27,153,Для детей
2,2184,87.0,2009-12-22,7.27,153,Для детей


In [6]:
from sklearn.preprocessing import LabelEncoder

# кодируем индексы пользователей
user_encoder = LabelEncoder()
user_encoder.fit(content_views.user_id)

# ереиндексация контента
content_views = content_views.assign(
    user_index = user_encoder.transform(content_views.user_id)
)

# кодируем индексы контента
item_encoder = LabelEncoder()
item_encoder.fit(content_views.content_id)

# нова переиндексация
content_views = content_views.assign(
    item_index = item_encoder.transform(content_views.content_id)
)


content_views.head()

,user_id,content_id,view_duration,view_ts,dt,platform,user_index,item_index
0,4649,52867,735,2019-03-18 20:40:57+03:00,2019-03-18,LG,802,22812
1,16,48800,361,2019-03-18 11:48:27+03:00,2019-03-18,LG,2,20399
2,5380,47146,268,2019-02-17 13:06:33+03:00,2019-02-17,LG,911,19628
3,4498,30191,297,2019-03-18 15:27:18+03:00,2019-03-18,LG,773,13517
4,4886,39349,302,2019-03-18 12:08:16+03:00,2019-03-18,LG,836,16959


In [7]:
from scipy.sparse import csr_matrix

num_users = content_views.user_index.max() + 1
num_items = content_views.item_index.max() + 1
num_interactions = content_views.shape[0]

user_item = csr_matrix(
    (np.ones(num_interactions),(content_views.user_index.values, content_views.item_index.values)),
    shape=(num_users, num_items)
)
print('sparsity: %.4f' % (num_interactions / (num_users * num_items)))

user_item

sparsity: 0.0091


<2000x27012 sparse matrix of type '<class 'numpy.float64'>'
	with 259994 stored elements in Compressed Sparse Row format>

In [8]:
from sklearn.model_selection import train_test_split

train_ids, test_ids = train_test_split(
    np.arange(start=0, stop=user_item.shape[0], step=1, dtype=np.uint32),
    test_size=0.2
)
print(
    "Размер обучающей выборки %d пользователей, размер валидационной выборки %d пользователей"
    % (train_ids.size, test_ids.size)
)



Размер обучающей выборки 1600 пользователей, размер валидационной выборки 400 пользователей


In [9]:
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)

# обучаемся только на тренировочной части пользователей
model_knn.fit(user_item[train_ids,:])

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

In [10]:
class ColaborativeFilteringKNNRecommender:
    def __init__(self, knn_model, user_item_matrix, num_neighbors):
        self.knn_model = knn_model
        self.user_item_matrix = user_item_matrix
        self.num_neighbors = num_neighbors
        self.top_recs = 50
    
    def make_recs(self, user_history: csr_matrix, top_recs: int):
        neighbors = model_knn.kneighbors(
            random_user_history,
            self.num_neighbors,
            return_distance=False
        )[0]
        full_recs = user_item[neighbors,:].max(axis=0)
        # рекомендации - это то, что насмотрели ближайшие соседи
        user_history_ids = user_history.nonzero()[1]
        # последовательность id того контента, который смотрели ближайшие соседи
        full_recs_ids = full_recs.nonzero()[1][:self.top_recs]
        # исключаем из рекомендаций то, что уже было у упользователя в историии
        success_recs = np.array([i for i in full_recs_ids if i in user_history_ids])
        print("Число успешных рекомендаций %d из %d" % (success_recs.size, top_recs))
        
        return np.array([i for i in full_recs_ids if i not in user_history_ids])[:10]


# объект рекомендателя
recommender = ColaborativeFilteringKNNRecommender(
    knn_model=model_knn,
    user_item_matrix=user_item,
    num_neighbors=10
)

In [11]:
# пример рекомендаций для случайного пользователя
random_user_index = 786
random_user_history = user_item.getrow(random_user_index).reshape(1, -1)

recs = recommender.make_recs(random_user_history, top_recs=10)
print('user_index %d, history: %s' % (random_user_index, random_user_history.nonzero()[1][:10]))
print('recommendations: %s' % recs)

Число успешных рекомендаций 4 из 10
user_index 786, history: [ 0  1  2  5  6  7  8 80 92 94]
recommendations: [  9  76 251 252 301 468 469 470 471 472]


In [12]:
# -- ВАШ КОД ТУТ --

from implicit.nearest_neighbours import CosineRecommender

cosine_recommender = CosineRecommender(K=50, num_threads=0)
cosine_recommender.fit(user_item[train_ids,:])

cosine_recommender.recommend(random_user_index, user_item[train_ids,:])
# ------------------

  0%|          | 0/1600 [00:00<?, ?it/s]

[(1510, 1.2214190707117298),
 (141, 1.0),
 (25, 1.0),
 (1487, 1.0),
 (981, 1.0),
 (1481, 1.0),
 (289, 1.0),
 (26, 0.9999999999999702),
 (1433, 0.9802296092526569),
 (1264, 0.9639458273010122)]

## Домашнее задание: Item to Item

Решите задачу c2c рекомендаций - вызовите метод `similar_items` для  *item_id=1*

In [13]:
# -- ВАШ КОД ТУТ --

cosine_recommender.similar_items(1)

# ------------------

[(1, 1.0),
 (693, 1.0),
 (1489, 0.023044900641260895),
 (704, 0.022599230735278437),
 (581, 0.019984019174435787),
 (136, 0.015021044203152742),
 (230, 0.0024420333880701473)]

### Домашнее задание: обучаем Implicit

Почитайте документацию по модулю implicit.als.AlternatingLeastSquares. Обучите ALS-рекомендатель и воспользуйтесь методом recommend для построения рекомендаций

In [15]:
# -- ВАШ КОД ТУТ --

from implicit.als import AlternatingLeastSquares

als_recommender = AlternatingLeastSquares()
als_recommender.fit(user_item[train_ids,:])

als_recommender.recommend(random_user_index, user_item[train_ids,:])
# -----------------

2021-04-21 17:59:21,727 | WARNING  | utils.py                 :26   | OpenBLAS detected. Its highly recommend to set the environment variable 'export OPENBLAS_NUM_THREADS=1' to disable its internal multithreading


  0%|          | 0/15 [00:00<?, ?it/s]

[(497, 0.91815084),
 (57, 0.7808006),
 (887, 0.21588716),
 (803, 0.0818508),
 (1109, 0.07203824),
 (236, 0.06840785),
 (499, 0.062426552),
 (500, 0.058115765),
 (1563, 0.05491588),
 (1015, 0.053727902)]

### Домашнее задание на метрики

Даны два вектора - истинная история пользователя и объекты, которые считает релеватными ваша модель

Вычислите

* precision
* recall
* precision@5


In [27]:
import numpy as np

user_interactions = [47315, 30004, 36322,  8942, 30820,  6086,  9126,   332, 16289,
       39106, 39335, 48506, 48654,  9234, 29935,  2678, 36202, 22636, 18007, 39328, 15414, 30016, 35601,
    58409, 21313,   386, 16303, 4397, 19644, 51887, 21659, 36325, 53030,  7764, 50266, 58734, 53419, 24121,
    50806, 36092,  8868, 28037, 36131, 13561, 16298, 27508, 41722, 30189, 46490,  2676, 43328, 781, 48397,
    41369, 39324, 36381, 39635, 27710, 47837, 28525, 12024, 56604, 41664, 37387, 48507, 413, 33526, 20059,
    49781, 56648, 16283, 50805, 34254, 39325, 59374, 22620,  8865, 27512, 13875, 30011,  7621,
    10544, 28076, 29716, 30054, 20490, 29466, 16852, 39363, 34250, 7024, 33541,   263, 21267, 25690, 23020,
    41368, 53414,  2681, 30201] 

user_recs = [
    50820, 27781, 36131, 50812, 36092, 12024, 59155, 30042, 15414, 19882, 21659, 27849, 39328, 34240, 2681,
    21267, 50126, 58560, 7764, 49781
]

# --- ВАШ КОД ТУТ ---

relevant_documents = set(user_recs)
retreived_documents = set(user_interactions)


recall =  len(relevant_documents & retreived_documents) / len(retreived_documents)
precision =  len(relevant_documents & retreived_documents) / len(relevant_documents) 


def average_precission_at(relevant: list, retreived: list, at: int) -> float:
    """Вроде бы правильно понял, но не уверен."""
    s = 0
    _at = at

    for item in relevant:
        if at == 0:
            break

        try:
            nidx = retreived.index(item) + 1
        except:
            continue
            
        _at -= 1
        s += 1/nidx
    
    return (1/at) * s

average_precision_at_5 = average_precission_at(user_recs, user_interactions, 5)

precision, recall, average_precision_at_5
# -------------------

(0.5, 0.1, 0.05183403900280742)